In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os

# Paths
data_dir = "/content/drive/MyDrive/Autism Images"
output_dir = "/content/drive/MyDrive/Autism Images Augmented"
os.makedirs(output_dir, exist_ok=True)

# Image size
IMG_SIZE = (224, 224)

# Keras ImageDataGenerator for offline augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=2,          # tiny rotation
    width_shift_range=0.02,    # tiny horizontal shift
    height_shift_range=0.02,   # tiny vertical shift
    brightness_range=[0.9,1.1], # slight brightness change
    zoom_range=0.02,           # tiny zoom
    fill_mode='nearest'
)

# Loop through original dataset
classes = [cls for cls in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, cls))]

num_aug_per_image = 5  # Generate 5 augmented images per original image

for cls in classes:
    class_input_dir = os.path.join(data_dir, cls)
    class_output_dir = os.path.join(output_dir, cls)
    os.makedirs(class_output_dir, exist_ok=True)

    for img_name in os.listdir(class_input_dir):
        img_path = os.path.join(class_input_dir, img_name)
        # Skip non-image files
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        img = load_img(img_path, target_size=IMG_SIZE)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)  # Keras expects 4D array

        i = 0
        for batch in datagen.flow(x, batch_size=1,
                                  save_to_dir=class_output_dir,
                                  save_prefix='aug',
                                  save_format='png'):
            i += 1
            if i >= num_aug_per_image:
                break